# Imports

In [78]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
from numpy import mean
from igraph import Graph as igraphGraph
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial.distance 


# Input

In [79]:
transformation = Transformation()

user_number_triangles = 500   #à diminuer si le process est trop long
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [80]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [81]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [82]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([0.1, 0.1, 0.1]).reshape(1, -1)  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = current_node - neigh_nodes                                                           # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [83]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[0.8046951  0.80516328 0.8046951  ... 0.74914404 1.08744362 0.74914404]
[0.73495107 0.73537867 0.73495107 ... 0.6842147  0.99319339 0.6842147 ]
[0.61554422 0.61560902 0.61554422 ... 0.60782656 0.65386268 0.60782656]
(5999,)


### Multinomial Sampling

In [84]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[0 0 0 ... 0 0 0]


In [85]:
target_number_point = min(len(graph._node), user_number_triangles*3)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]     # Take the k ("target_number_points") largest values given by the multinomial sampling 
list_k_nodes = np.array(list(graph._node.keys()))[index_k_nodes]                                # Take the selected nodes (list of list of 3)
list_k_nodes[:5]

array([[89.50307  ,  2.3044074, 90.6184   ],
       [77.309    ,  7.328478 , 84.37807  ],
       [88.439125 ,  2.3044074, 92.08279  ],
       [87.22794  ,  2.3044074, 93.42794  ],
       [88.990265 ,  2.3044074, 91.364525 ]], dtype=float32)

# KNN extended graph

In [86]:
"""
_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points

extended_graph = nx.Graph()
for index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node
    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates
    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)
        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples
        extended_graph.add_edge(*edge)                                      # Add the edge to the graph

transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)
"""

"\n_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points\n\nextended_graph = nx.Graph()\nfor index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node\n    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates\n    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)\n        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples\n        extended_graph.add_edge(*edge)                                      # Add the edge to the graph\n\ntransformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)\n"

In [87]:
def knn_and_extended_graph(list_k_nodes_i, k_neighbors=15):
    """
    Perform k-NN on the given points and construct an extended graph.

    Parameters:
    - list_k_nodes_i: array-like
        List of coordinates of the selected points.
    - k_neighbors: int, optional (default=15)
        Number of neighbors for k-NN.

    Returns:
    - extended_graph_i: networkx.Graph
        Extended graph.
    """

    # Perform k-NN on the selected points
    _, indices = NearestNeighbors(n_neighbors=k_neighbors).fit(list_k_nodes_i).kneighbors(list_k_nodes_i)

    # Construct the extended graph
    extended_graph_i = nx.Graph()
    for index_poly, poly in enumerate(indices):
        current_node = tuple(list_k_nodes_i[poly[0]])
        for index_other_node in poly[1:]:
            edge = current_node, tuple(list_k_nodes_i[index_other_node])
            extended_graph_i.add_edge(*edge)

    return extended_graph_i

extended_graph = knn_and_extended_graph(list_k_nodes)
transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 1500
Number of edges: 11773


# Edge Predictor

In [88]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(1500, 64)

In [89]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [90]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

In [91]:
# S = S*np.random.choice([0, 1], size=S.shape)      # Add a random mask to emulate the 'sparse'

# Face Candidates

#### Inputs

In [92]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 30)	1
  (0, 185)	1
  (0, 186)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (1, 10)	1
  :	:
  (1498, 1217)	1
  (1498, 1218)	1
  (1498, 1219)	1
  (1498, 1220)	1
  (1498, 1239)	1
  (1498, 1485)	1
  (1498, 1494)	1
  (1498, 1497)	1
  (1498, 1499)	1
  (1499, 1220)	1
  (1499, 1406)	1
  (1499, 1407)	1
  (1499, 1408)	1
  (1499, 1411)	1
  (1499, 1412)	1
  (1499, 1486)	1
  (1499, 1487)	1
  (1499, 1488)	1
  (1499, 1489)	1
  (1499, 1490)	1
  (1499, 1491)	1
  (1499, 1494)	1
  (1499, 1495)	1
  (1499, 1496)	1
  (1499, 1498)	1
[[0.05925598 0.05920983 0.05920983 ... 0.05976973 0.05969106 0.05978231]
 [1.32504849 0.06678316 0.06678316 ... 0.06690603 0.06689326 0.06690789]
 [1.32474397 0.51120261 0.06643015 ... 0.06597121 0.06604706 0.06595864]
 ...
 [1.84693181 1.57644792 1.60684758 ... 0.03326687 0.0372411

In [93]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0
A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T
A_s = A_s/A_s.max()                                         # Normalize

In [94]:
print(A_s)  # symmétrique
print(A_s.shape)

[[3.06784151e-15 3.49109991e-15 3.46791260e-15 ... 2.57379991e-13
  2.08081700e-13 2.71322718e-13]
 [3.49109991e-15 3.97277271e-15 3.96748972e-15 ... 2.87044449e-13
  2.32571659e-13 3.01931021e-13]
 [3.46791260e-15 3.96748972e-15 3.96293873e-15 ... 2.80212105e-13
  2.27497622e-13 2.94098423e-13]
 ...
 [2.57379991e-13 2.87044449e-13 2.80212105e-13 ... 7.42791627e-11
  5.43093427e-11 7.64341152e-11]
 [2.08081700e-13 2.32571659e-13 2.27497622e-13 ... 5.43093427e-11
  4.00370422e-11 5.61002212e-11]
 [2.71322718e-13 3.01931021e-13 2.94098423e-13 ... 7.64341152e-11
  5.61002212e-11 8.89591114e-11]]
(1500, 1500)


# Face Classifier

### TriConv

#### Inputs

In [95]:
nodes = list(extended_graph.nodes())
edges = list(extended_graph.edges())

igraph_g = igraphGraph(directed=extended_graph.is_directed())

igraph_g.add_vertices(nodes)
igraph_g.add_edges([(nodes.index(u), nodes.index(v)) for u, v in edges])
print(igraph_g.summary())

IGRAPH UN-- 1500 11773 -- 
+ attr: name (v)


In [96]:
triangles_ids_igraph = np.array(igraph_g.cliques(min=3, max=3))
print(triangles_ids_igraph[:3])
print(len(triangles_ids_igraph))

[[ 687  923  924]
 [1396 1398 1472]
 [1076 1145 1255]]
32719


In [97]:
triangles = np.array(igraph_g.vs['name'])[triangles_ids_igraph]   #les triangles
# triangles = [list(map(tuple, row)) for row in triangles.tolist()]
print(triangles[:3])
print(len(triangles))

[[[ 75.08098    -1.9813722   3.5890484]
  [ 75.16284    -2.6241417   3.4342492]
  [ 75.20018    -2.6241417   3.4532745]]

 [[ 81.59701    -4.7321377 -11.017121 ]
  [ 81.85278    -3.5589852 -11.591602 ]
  [ 82.363434   -4.7321377 -10.651554 ]]

 [[ 75.85237     5.0095077  82.51254  ]
  [ 75.24483     5.0095077  82.74576  ]
  [ 75.78252     5.0095077  82.55535  ]]]
32719


In [98]:
# Extract indices for each triangle
i, j, k = triangles_ids_igraph.T

# Extract probabilities using advanced indexing
A_s_ij = A_s[i, j]
A_s_ik = A_s[i, k]
A_s_jk = A_s[j, k]

# Calculate the barycenter probabilities
p_init = np.zeros(len(igraph_g.vs))
p_init[i] = (A_s_ij + A_s_ik + A_s_jk) / 3

#### Calculate barycenter

In [99]:
barycenters = np.mean(triangles, axis=1)
print(barycenters.shape)
print(barycenters[:3])

(32719, 3)
[[ 75.148      -2.4098852   3.4921906]
 [ 81.937744   -4.341087  -11.086759 ]
 [ 75.62657     5.0095077  82.60455  ]]


#### KNN Tri

In [100]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)          # Find k='number_neigh_tri' neighbors of each triangles based on theirs barycenters

Diff Barycenters

In [101]:
# Calculates the difference between each barycenters and their n='number_neigh_tri'-1 neighbors
barycenters_diff = np.subtract(barycenters[indices_neigh_tri[:, 0]][:, np.newaxis], barycenters[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(32719, 19, 3)


#### calculate e norm matrix

In [102]:
diff_vectors = list()

for triangle in triangles:
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))    #Calculate the vectors for each edge of each triangles
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])

diff_vectors = np.array(diff_vectors)
print(diff_vectors.shape)
print(diff_vectors[:2])

(32719, 3)
[[0.6661959  0.66767967 0.04190595]
 [1.3310658  0.84914565 1.5876843 ]]


#### Calculate r

In [103]:
max_diff_vectors = diff_vectors.max(axis=1)       # calculate t_n_max
min_diff_vectors = diff_vectors.min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(32719,)

In [104]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(32719, 19)
(32719, 19)


In [105]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))

r_matrix[:, :, 0]   = min_diff_vectors_diff
r_matrix[:, :, 1]   = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(32719, 19, 5)


#### Calculate f

In [106]:
# Multi Layer Perceptron (MLP) * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

1.0000000000000002
(1500,)
[0.0006656 0.0006656 0.0006656 ... 0.0006656 0.0006656 0.0006656]


C:\Users\Arthur\AppData\Local\Temp\ipykernel_4688\474768578.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles


# Simplified Mesh

In [107]:
selected_triangles_indexes = np.argpartition(final_scores, -int(target_number_point/3))[-int(target_number_point/3):] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)
print(selected_triangles[:1])

(500, 3, 3)
[[[83.10968   -4.7321377 95.246376 ]
  [82.92011   -5.2797017 94.91804  ]
  [83.39875   -5.7970243 94.13294  ]]]


In [108]:
simplified_final_graph = nx.Graph()
for index_poly, poly in enumerate(selected_triangles):
    for index_current_node in range(len(poly)):
        current_node = tuple(poly[index_current_node])
        for index_other_node in range(index_current_node+1, len(poly)):
            edge = current_node, tuple(poly[index_other_node])
            simplified_final_graph.add_edge(*edge)
            # if attribute do not exists
            if len(simplified_final_graph.nodes[current_node])==0:
                simplified_final_graph.nodes[current_node]['index_triangle'] = set()
            simplified_final_graph.nodes[current_node]['index_triangle'].add(index_poly)
            if len(simplified_final_graph.nodes[tuple(poly[index_other_node])])==0:
                simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
            simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
            
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 438
Number of edges: 905


In [112]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

#Affichage
# transformation.mesh_to_display_vtk(mesh_data)
# transformation.mesh_to_display_vtk(simplified_final_mesh)

# Fonctions Loss

## Probabilistic Chamfer distance

Récupération des points de l'espace de départ P

In [139]:
keys_input = np.array(graph)
print("Les points avant K-NN : ", keys_input)
print("dim points avant K-NN : ", len(keys_input))  #return 5999

Les points avant K-NN :  [[75.0252     -0.66553295  3.8       ]
 [75.01418    -0.49935842 16.7       ]
 [75.          0.          3.8       ]
 ...
 [51.         -8.57404    -6.9813724 ]
 [39.         -8.699333   -6.327254  ]
 [51.         -8.699333   -6.327254  ]]
dim points avant K-NN :  5999


Récupération des points de l'espace de départ Ps

In [140]:
keys_output = np.array(extended_graph)
print("sampled points : ", keys_output)
print("dim sample points : ", len(keys_output))  #return: 50

sampled points :  [[89.50307    2.3044074 90.6184   ]
 [89.62883    1.6554987 90.70007  ]
 [89.97611    2.3044074 89.84646  ]
 ...
 [88.219635   6.7292814 -2.0857615]
 [87.42555    7.138877  -2.5311363]
 [84.058      7.8332825 -5.2580013]]
dim sample points :  1500


Distance Matrix

In [141]:
def squared_euclidean_distances(x, y):
    x_squared_sum = np.sum(x**2, axis=1)
    y_squared_sum = np.sum(y**2, axis=1)
    xy_dot_product = np.dot(y, x.T)

    distances = np.outer(y_squared_sum, np.ones_like(x_squared_sum)) - 2 * xy_dot_product + np.outer(np.ones_like(y_squared_sum), x_squared_sum)

    return distances

distances_matrix = squared_euclidean_distances(keys_input, keys_output)
print("Squared Euclidean Distances Matrix Shape:", distances_matrix.shape)
print("Distances Matrix:\n", distances_matrix)




Squared Euclidean Distances Matrix Shape: (1500, 5999)
Distances Matrix:
 [[ 7755.863    5681.7183   7753.0825  ... 11126.543   12070.102
  11002.028  ]
 [ 7770.275    5694.2417   7768.3657  ... 11138.494   12084.803
  11013.711  ]
 [ 7636.3433   5582.1245   7633.5864  ... 11013.106   11969.029
  10889.603  ]
 ...
 [  263.41846   579.5425    254.68408 ...  1643.4607   2678.6055
   1641.3337 ]
 [  254.75977   582.2217    245.44043 ...  1593.5208   2610.2935
   1592.0803 ]
 [  235.86865   633.37744   225.45459 ...  1365.0017   2304.6934
   1367.3025 ]]


In [ ]:
# Calculer le min des x de distances_matrix
def min_pour_x_list(distances_matrix, x_in):
    return np.min(distances_matrix[:, x_in])

# Calculer le min des y de distances_matrix
def min_pour_y_list(distances_matrix, y_out):
    return np.min(distances_matrix[y_out, :])


In [ ]:
def d_P_Ps(keys_x, keys_y, p_y):
    """
    Calculate the Probabilistic Chamfer distance between two sets of points.

    Parameters:
    - keys_x (list): The input vertex set, a list of tuples.
    - keys_y (list): The sampled points, a list of tuples.
    - p_y (list): Their respective probabilities, a weight associated with each point in keys_y.

    Returns:
        chamfer_distance (float): The Probabilistic Chamfer distance between the two sets of points.
    """
    min_for_x = [p_y[i] * min_pour_x_list(distances_matrix, i) for i in range(len(keys_x))]
    min_for_y = [p_y[j] * min_pour_y_list(distances_matrix, j) for j in range(len(keys_y))]
    
    sum_for_y = np.sum(min_for_y)
    sum_for_x = np.sum(min_for_x)

    chamfer_distance = sum_for_y + sum_for_x
    
    return chamfer_distance


In [ ]:
print("PROBA : ", normalized_inclusion_score)

PROBA :  [0.00020681 0.00020684 0.00020681 ... 0.00020336 0.00022335 0.00020336]


In [ ]:
prob_chamfer_dist = d_P_Ps(keys_input, keys_output, normalized_inclusion_score)
print("Chamfer Distance:", prob_chamfer_dist)

Chamfer Distance: 0.0


## Probabilistic Surfaces Distance

barycentres b et b_hat

In [39]:
b = np.array(list(graph._node.keys()))
print("Barycentre b: ", b)

Barycentre b:  [[75.0252     -0.66553295  3.8       ]
 [75.01418    -0.49935842 16.7       ]
 [75.          0.          3.8       ]
 ...
 [51.         -8.57404    -6.9813724 ]
 [39.         -8.699333   -6.327254  ]
 [51.         -8.699333   -6.327254  ]]


In [40]:
b_hat = np.array(list(extended_graph._node.keys()))
print("B_HAT : ", b_hat)

B_HAT :  [[ 75.0252      -0.66553295   3.8       ]
 [ 75.02517     -0.66553295   3.798681  ]
 [ 75.02506     -0.66553295   3.7973657 ]
 ...
 [ 51.          -6.3966537  -11.043411  ]
 [ 39.          -6.8353896  -10.542333  ]
 [ 51.          -6.8353896  -10.542333  ]]


test matrice des normes euclidiennes entre b et b_hat

In [41]:
from scipy.spatial.distance import cdist, pdist, squareform
def matrix_distance_euclidienne_barycentres(barycentre_b, barycentre_b_hat):
    """
    Calculate distances between barycenters b et b_hat after the k-NN process.

    Parameters:
    - b_hat: array-like, shape (M, 3)
        Barycenters after k-NN.
    - number_neigh: int, optional (default=3)
        Number of neighbors for k-NN.

    Returns:
    - distances_after: array-like, shape (M, M)
        Matrix of distances between barycenters after the k-NN process.
    """
    return cdist(barycentre_b, barycentre_b_hat, metric='sqeuclidean')
    
distances = matrix_distance_euclidienne_barycentres(b, b_hat)
print("Distances euclidiennes entre les barycentres b et b_hat :\n", distances)


Distances euclidiennes entre les barycentres b et b_hat :
 [[0.00000000e+00 1.74051183e-06 6.95832665e-06 ... 8.30382831e+02
  1.54158466e+03 8.20979866e+02]
 [1.66437756e+02 1.66471786e+02 1.66505725e+02 ... 1.38115599e+03
  2.07931140e+03 1.35897101e+03]
 [4.43569137e-01 4.43569340e-01 4.43569174e-01 ... 8.37244034e+02
  1.54842506e+03 8.28425056e+02]
 ...
 [7.55992709e+02 7.55962805e+02 7.55929315e+02 ... 2.12411731e+01
  1.59703345e+02 1.57033448e+01]
 [1.46491824e+03 1.46488933e+03 1.46485500e+03 ... 1.71544474e+02
  2.12411752e+01 1.65241175e+02]
 [7.44313445e+02 7.44285266e+02 7.44253497e+02 ... 2.75444740e+01
  1.65241175e+02 2.12411752e+01]]


matrice sqeuclidean b

In [42]:
def normes_euclidiennes_carrees_b(bar_b):
    # Calculer la matrice des normes euclidiennes au carré
    normes_carrees = np.sum(bar_b**2, axis=1)

    return normes_carrees

matrix_b = normes_euclidiennes_carrees_b(b)
print(matrix_b)

[5643.6636 5906.267  5639.44   ... 2723.2537 1636.7125 2716.7126]


In [43]:
def matrix_distances_after_knn(b_hat_n, number_neigh=3):
    """
    Calculate distances between barycenters after the k-NN process.

    Parameters:
    - b_hat: array-like, shape (M, 3)
        Barycenters after k-NN.
    - number_neigh: int, optional (default=3)
        Number of neighbors for k-NN.

    Returns:
    - distances_after: array-like, shape (M, M)
        Matrix of distances between barycenters after the k-NN process.
    """

    # Connect the extended graph using the k-NN process
    extended_graph = knn_and_extended_graph(b_hat_n, number_neigh)

    # Extract barycenters after the k-NN process
    b_hat_extended = np.array(list(extended_graph._node.keys()))

    # Calculate distances after k-NN
    distances_after = cdist(b_hat_extended, b_hat_extended)

    return distances_after

# Example usage with b_hat
distances_after = matrix_distances_after_knn(b_hat)

# Print the results
print("Distances After k-NN:\n", distances_after)

Distances After k-NN:
 [[0.00000000e+00 1.31928459e-03 2.63786403e-03 ... 2.89693862e+01
  3.93825803e+01 2.88163639e+01]
 [1.31928459e-03 0.00000000e+00 1.31968491e-03 ... 2.89686639e+01
  3.93820553e+01 2.88156591e+01]
 [2.63786403e-03 1.31968491e-03 0.00000000e+00 ... 2.89678802e+01
  3.93814618e+01 2.88148926e+01]
 ...
 [2.89693862e+01 2.89686639e+01 2.89678802e+01 ... 0.00000000e+00
  1.20184679e+01 6.66010378e-01]
 [3.93825803e+01 3.93820553e+01 3.93814618e+01 ... 1.20184679e+01
  0.00000000e+00 1.20000000e+01]
 [2.88163639e+01 2.88156591e+01 2.88148926e+01 ... 6.66010378e-01
  1.20000000e+01 0.00000000e+00]]


In [44]:
def probabilistic_surface_distance_extended(b_i, b_hat_i, p_b_hat_i):
    """
    Calculate Probabilistic Surfaces Distance 

    Parameters:
    - b: array-like, shape (N, 3)
        Barycenters of the ground truth surface.
    - b_hat: array-like, shape (M, 3)
        Barycenters of the generated triangles.
    - p_b_hat: array-like, shape (M,)
        Probabilities corresponding to the generated triangles.

    Returns:
    - d_S_Ss: float
        Probabilistic Surfaces Distance.
    """
    # Debugging print statements
    print("b shape:", b_i.shape)
    print("b_hat shape:", b_hat_i.shape)
    print("p_b_hat shape:", p_b_hat_i.shape)

    p_b_hat_i = normalized_inclusion_score.reshape((-1, 1))
    dist_matrixs_1= distances_after - matrix_b
    #dist_matrixs_2= matrix_b - distances_after

    d_S_Ss = np.sum(p_b_hat_i * np.min(np.sum(dist_matrixs_1)**2))

    

    return d_S_Ss

result_distance = probabilistic_surface_distance_extended(b, b_hat, normalized_inclusion_score)
print("Probabilistic Surfaces Distance:", result_distance)

b shape: (5999, 3)
b_hat shape: (5999, 3)
p_b_hat shape: (5999,)
Probabilistic Surfaces Distance: 1.2801823949999125e+23


## Triangle Collision Loss

In [ ]:
def triangle_collision_loss(generated_triangles, probabilities):
    """
    Calculate triangle collision loss to penalize triangles that penetrate each other.

    Parameters:
    - generated_triangles (numpy.ndarray): Array representing the generated triangles.
    - probabilities (numpy.ndarray): Array of probabilities for each generated triangle.

    Returns:
    - float: Triangle collision loss.
    """
    num_triangles = len(generated_triangles)

    # Create a 3D array to represent all combinations of triangle pairs
    triangle_pairs_i, triangle_pairs_j = np.meshgrid(range(num_triangles), range(num_triangles), indexing='ij')

    # Filter out pairs where i is not equal to j
    valid_pairs_mask = triangle_pairs_i != triangle_pairs_j

    # Extract triangles and probabilities for each pair
    triangles_i = generated_triangles[triangle_pairs_i[valid_pairs_mask]]
    triangles_j = generated_triangles[triangle_pairs_j[valid_pairs_mask]]
    probs_i = probabilities[triangle_pairs_i[valid_pairs_mask]]

    # Calculate the plane normals and distances for all triangles
    plane_normals = np.cross(triangles_i[:, 1] - triangles_i[:, 0], triangles_i[:, 2] - triangles_i[:, 0])

    # Calculate the edge vectors for all edges of triangles_j
    edge_vectors = triangles_j[:, [1, 2, 0], :] - triangles_j[:, [0, 1, 2], :]

    # Calculate the dot product of edge vectors and plane normals
    dot_products = np.sum(np.cross(edge_vectors, plane_normals[:, np.newaxis, :]) * triangles_i[:, 0], axis=2)

    # Count the number of faces penetrated for each triangle
    faces_penetrated = np.sum(dot_products < 0, axis=1)

    # Calculate the loss for each triangle
    loss_per_triangle = probs_i * faces_penetrated

    # Sum up the losses and normalize by the number of triangles
    triangle_losses = np.sum(loss_per_triangle) / num_triangles

    return triangle_losses

